In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.types import Row
from pyspark.sql import SQLContext

In [2]:
spark = SparkContext(master='local', appName='dataframes')
sql = SQLContext(spark)

23/04/04 05:58:14 WARN Utils: Your hostname, PRRS-DCARVAJAL resolves to a loopback address: 127.0.1.1; using 172.30.122.249 instead (on interface eth0)
23/04/04 05:58:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 05:58:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/daielchom/spark/python/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
!ls ./files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista.csv	 evento.csv	      paises.csv
deportista2.csv  juegos.csv	      resultados.csv


In [4]:
!head -n 5 ./files/juegos.csv

,nombre_juego,annio,temporada,ciudad
1,1896 Verano,1896,Verano,Athina
2,1900 Verano,1900,Verano,Paris
3,1904 Verano,1904,Verano,St. Louis
4,1906 Verano,1906,Verano,Athina


In [5]:
path = './files/'

In [6]:
game_schema = StructType([StructField('juego_id', IntegerType(),False),
                          StructField('annio', StringType(),False),
                          StructField('temporada', StringType(),False),
                          StructField('ciudad', StringType(),False)])
df_game = sql.read.schema(game_schema).option('header','true').csv(path+'/juegos.csv')

In [7]:
df_game.show(4)

23/04/04 05:58:30 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 5, schema size: 4
CSV file: file:///home/daielchom/projects/spark_notebooks/files/juegos.csv


+--------+-----------+---------+------+
|juego_id|      annio|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
+--------+-----------+---------+------+
only showing top 4 rows



### Class 14

In [10]:
deportista_olimpico_rdd = spark.textFile(path+'deportista.csv').map(lambda l:l.split(','))
deportista_olimpico_rdd.top(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['9999', 'Ferenc Bene', '1', '19', '170', '70', '477'],
 ['9998', 'Kurt Bendlin', '1', '25', '178', '91', '1150'],
 ['9997', 'Ole Christian Bendixen', '1', '25', '180', '75', '742'],
 ['9996', 'Finn Berg Bendixen', '1', '23', '185', '74', '742']]

In [11]:
def remove_header(index, iterator):
    return iter(list(iterator)[1:])

In [12]:
deportista_olimpico_rdd = deportista_olimpico_rdd.mapPartitionsWithIndex(remove_header)

In [13]:
deportista_olimpico_rdd.top(5)

[['9999', 'Ferenc Bene', '1', '19', '170', '70', '477'],
 ['9998', 'Kurt Bendlin', '1', '25', '178', '91', '1150'],
 ['9997', 'Ole Christian Bendixen', '1', '25', '180', '75', '742'],
 ['9996', 'Finn Berg Bendixen', '1', '23', '185', '74', '742'],
 ['9995', 'HansJrg Bendiner', '1', '23', '184', '82', '1019']]

In [15]:
deportista_olimpico_rdd = deportista_olimpico_rdd.map(lambda l:(int(l[0]), l[1], int(l[2]), int(l[3]), int(l[4]), float(l[5]), int(l[6])))

In [16]:
schema = StructType([StructField('deportista_id', IntegerType(),False),
                     StructField('nombre', StringType(),False),
                     StructField('genero', IntegerType(),False),
                     StructField('edad', IntegerType(),False),
                     StructField('altura', IntegerType(),False),
                     StructField('peso', FloatType(),False),
                     StructField('equipo_id', IntegerType(),False),
                    ])

In [18]:
df_sport = sql.createDataFrame(deportista_olimpico_rdd, schema)
df_sport.show(5)

+-------------+--------------------+------+----+------+----+---------+
|deportista_id|              nombre|genero|edad|altura|peso|equipo_id|
+-------------+--------------------+------+----+------+----+---------+
|            1|           A Dijiang|     1|  24|   180|80.0|      199|
|            2|            A Lamusi|     1|  23|   170|60.0|      199|
|            3| Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
|            4|Edgar Lindenau Aabye|     1|  34|     0| 0.0|      278|
|            5|Christine Jacoba ...|     2|  21|   185|82.0|      705|
+-------------+--------------------+------+----+------+----+---------+
only showing top 5 rows



In [19]:
spark.stop()